In [1]:
# Web scraping stuff
import requests
!conda install -c anaconda beautifulsoup4 --yes
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\movermye\AppData\Local\Continuum\anaconda3\envs\base_astro

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.2       |           py36_0         163 KB  anaconda
    ca-certificates-2019.11.27 |                0         163 KB  anaconda
    certifi-2019.11.28         |           py36_0         157 KB  anaconda
    conda-4.8.1                |           py36_0         3.1 MB  anaconda
    openssl-1.0.2s             |       he774522_0         5.8 MB  anaconda
    soupsieve-1.9.5            |           py36_0          61 KB  anaconda
    ------------------------------------------------------------
                                           Total:         9.4 MB

The following NEW packages will be INSTALLED:

  soupsieve          anaco

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


ModuleNotFoundError: No module named 'geopy'

# Pulling Postal Data from Wikipedia
using this tutorial: [bs4](https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722)

## Pulling from wiki

In [26]:
website_url_str = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
website_url = requests.get(website_url_str).text

#soup = BeautifulSoup(website_url,'lxml')
soup = BeautifulSoup(website_url,'lxml')

## Creating raw table

In [112]:
my_table = soup.find('table',{'class':'wikitable sortable'})

table_contents = my_table.find_all('td')
col = 0
post_codes = []
boroughs = []
neighborhoods = []
for tag in table_contents:
    if col == 0:
        post_codes.append(tag.get_text())
    elif col == 1:
        boroughs.append(tag.get_text())
    elif col == 2:
        neighborhoods.append(tag.get_text().strip())
    else:
        print('something is wrong')
        
    col = col + 1
    
    if col >= 3:
        col = 0
        
df = pd.DataFrame()
df['PostalCode'] = post_codes
df['Boroughs'] = boroughs
df['Neighborhoods'] = neighborhoods
    
df.head(15)

,PostalCode,Boroughs,Neighborhoods
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


## Cleaning up the table

In [113]:
# drop rows with unassigned boroughs
df.Boroughs = df['Boroughs'].replace('Not assigned', np.nan)
df.dropna(subset=['Boroughs'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(15)

,PostalCode,Boroughs,Neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [114]:
# fill missing neighborhood values with Borough value on same row
df.Neighborhoods = df.Neighborhoods.replace('Not assigned', np.nan)
df.Neighborhoods.fillna(df.Boroughs, inplace=True)
df.head(15)

,PostalCode,Boroughs,Neighborhoods
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [121]:
# combining Neighborhood text into comma seperated strings for repeated Postal Codes
df = df.groupby(['PostalCode', 'Boroughs'])['Neighborhoods'].apply(', '.join).reset_index()
df.head(15)

,PostalCode,Boroughs,Neighborhoods
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


# Final shape of clean table

In [122]:
df.shape

(103, 3)